## Data Preprocessing

In [4]:
# Importing Necessary Packages
import warnings
warnings.filterwarnings("ignore")

import math
import time
from datetime import date, timedelta, datetime
import numpy as np
import pandas as pd
import statistics as st
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import datetime
from pandas import Series,DataFrame

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline

## Setting Logging

In [5]:
import logging
import logging.handlers
import os
 
handler = logging.handlers.WatchedFileHandler(
    os.environ.get("LOGFILE", "../logs/prediction.log"))
formatter = logging.Formatter(logging.BASIC_FORMAT)
handler.setFormatter(formatter)
root = logging.getLogger()
root.setLevel(os.environ.get("LOGLEVEL", "INFO"))
root.addHandler(handler)
logging.info("Testing Loggings") 
try:
    exit(main())
except Exception:
    logging.exception("Exception in main()")
    exit(1)

## Creating Dataframe Class, and object

In [6]:
# Class to load the data
class FetchData():
    def __init__(self):
        self.dfdict = {}
        self.dfdict['train'] = self.get_train_data()
        self.dfdict['test'] = self.get_test_data()
        self.dfdict['sample'] = self.get_sample_data()
        self.dfdict['store'] = self.get_store_data()
        
    def get_train_data(self,name='train'):
        filename = f'../data/{name}.csv'
        try:
            df = pd.read_csv(filename)
            logging.info(f"{name} Dataset read successfully")
            return df

        except Exception as e:
            logging.exception(f" Exception occured in reading dataset, {e}")
            return None
    
    def get_test_data(self,name='test'):
        filename = f'../data/{name}.csv'        
        try:
            df = pd.read_csv(filename)
            logging.info(f"{name} Dataset read successfully")
            return df   

        except Exception as e:
            logging.exception(f" Exception occured in reading dataset, {e}")
            return None
        
    def get_store_data(self,name='store'):
        filename = f'../data/{name}.csv'        
        try:
            df = pd.read_csv(filename)
            logging.info(f"{name} Dataset read successfully") 
            return df  

        except Exception as e:
            logging.exception(f" Exception occured in reading dataset, {e}")
            return None 
    
    def get_sample_data(self,name='sample_submission'):
        filename = f'../data/{name}.csv'        
        try:
            df = pd.read_csv(filename)
            logging.info(f"{name} Dataset read successfully")  
            return df  

        except Exception as e:
            logging.exception(f" Exception occured in reading dataset, {e}")
            return None 
    
    def show_sample_data(self,dataset='train', sample=5):
        try:
            df = dataset.head(sample)
            return df  

        except Exception as e:
            logging.exception(f" Exception occured in getting sample data of a dataset, {e}")
            return None

    def find_null_values(self,dataset='train'):
        try:
            df = dataset.isnull().sum()
            logging.info("Getting Null values, Execution successfuly")
            return df  

        except Exception as e:
            logging.exception(f" Exception in getting Null values, {e}")
            return None

    def get_start_end_date(self, dataset_name='train', date_column='Date'):
        try:
            start_date = dataset_name[date_column].min()
            end_date = dataset_name[date_column].max()
            logging.info("Getting start and End date successfully")
            return start_date, end_date

        except Exception as e:
            logging.exception(f"Exception in getting start and end date, {e}")

            return None, None

    def join_dataset(self, dataset='train', dataset1='store'):
        if dataset.Store.nunique() == dataset1.Store.nunique():
            try:
                df_combined = dataset.merge(dataset1, how='left', left_on=dataset.Store, right_on=dataset1.Store)
                df_combined.drop(['key_0', 'Store_y'], axis=1, inplace=True)
                df_combined = df_combined.rename(columns={'Store_x':'Store'})
                logging.info(f" Joining {dataset} and {dataset1} datasets successfully")

                return df_combined.shape, df_combined
            
            except Exception as e:
                logging.debug(f"Exception in Joining {dataset} and {dataset1} datasets, {e}")
                return None, None

        else:
            logging.error("The values in the dataset are not compartible")
            print("The values in the dataset are not compartible")

    def add_day_month_year_to_dataset(self, dataset, column_list = ['day','month','year']):
        try:
            dataset.Date = pd.to_datetime(dataset.Date)
            for column in column_list:
                dataset[column] = dataset.Date.dt.column
                logging.info(f"Adding {column} column to dataset successfully")
                return dataset

        except Exception as e:
            logging.exception(f"Exception occured in Adding columns in dataset, Exception:{e}")
             
            return None

In [7]:
#  Creating FetchData Object
data = FetchData()

### Loading Datasets

In [8]:
try:
    train_data = data.get_train_data("train")
    store_data = data.get_train_data("store")
    test_data = data.get_train_data("test")
    sample_submission_data = data.get_train_data("sample_submission")

except Exception as e:
    logging.exception(f"Exception occured in loading dataset, Exception:{e}")

In [9]:
print(f" Train Data with shape of: {train_data.shape}")
print(train_data.info())
print(f" Store Data with sape of {store_data.shape}")
print(store_data.info())
print(f" Test Data with sape of {test_data.shape}")
print(test_data.info())
print(f" Sample Submission with sape of {sample_submission_data.shape}")
print(sample_submission_data.info())

 Train Data with shape of: (1017209, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB
None
 Store Data with sape of (1115, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-n

### It can be seen that, store dataset has many missing values, and test dataset has 11 missing value

## Handling 'NA' values
###  - Store dataset

In [10]:
store_data.isna().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [11]:
# Sorting the dataset by Competition Distance, because competition distance has less 'NA' values
store_data.sort_values(by='CompetitionDistance')

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
515,516,a,c,20.0,NaN,NaN,1,35.0,2010.0,"Mar,Jun,Sept,Dec"
881,882,a,a,30.0,4.0,2013.0,0,NaN,NaN,NaN
1007,1008,a,c,30.0,9.0,2010.0,0,NaN,NaN,NaN
620,621,a,a,30.0,7.0,2002.0,0,NaN,NaN,NaN
987,988,a,a,30.0,11.0,2012.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
121,122,a,c,58260.0,4.0,2013.0,0,NaN,NaN,NaN
452,453,a,c,75860.0,NaN,NaN,0,NaN,NaN,NaN
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN


### Handling the CompetitionDistance 'NA' values by filling with mean

In [12]:
store_data.CompetitionDistance.fillna(store_data.CompetitionDistance.mean(),inplace=True)
# store_data.sort_values(by='CompetitionDistance')

### Dealing with 'CompetitionOpenSinceMonth, CompetitionOpenSinceYear' NA values

In [13]:
store_data[store_data['CompetitionOpenSinceYear'].isna() == True].describe()


,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,354.000000,354.000000,0.0,0.0,354.000000,206.000000,206.000000
mean,560.584746,5430.634755,NaN,NaN,0.581921,25.480583,2011.567961
std,321.188639,7375.534157,NaN,NaN,0.493941,14.858261,1.638781
min,12.000000,20.000000,NaN,NaN,0.000000,1.000000,2009.000000
25%,277.500000,647.500000,NaN,NaN,0.000000,13.000000,2010.000000
50%,529.000000,2560.000000,NaN,NaN,1.000000,27.000000,2012.000000
75%,852.500000,7877.500000,NaN,NaN,1.000000,40.000000,2013.000000
max,1115.000000,75860.000000,NaN,NaN,1.000000,50.000000,2015.000000


### It can be seen that, both 'CompetitionOpenSinceMonth, CompetitionOpenSinceYear' are not-continuous data, therefore 'NA' will be filled with Mode

In [14]:
store_data.CompetitionOpenSinceMonth.fillna(store_data.CompetitionOpenSinceMonth.mode()[0],inplace=True)
store_data.CompetitionOpenSinceYear.fillna(store_data.CompetitionOpenSinceYear.mode()[0],inplace=True)
# store_data.sort_values(by='CompetitionOpenSinceYear')

### Dealing with Promotions columns

### From the above table it can be seen there is relation between Promo2 and 'Promo2SinceWeek, Promo2SinceYear, PromoInterval', when Promo2 is 0, all other promo columns are NaN
### Verifying this hypothesis

In [15]:
store_data.query('PromoInterval.isnull() & Promo2 == 0', engine='python')


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1107,1108,a,a,540.0,4.0,2004.0,0,NaN,NaN,NaN
1109,1110,c,c,900.0,9.0,2010.0,0,NaN,NaN,NaN
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,9.0,2013.0,0,NaN,NaN,NaN


### Therefore, replacing all 'NaN' values of promo coulumns with 0

In [16]:
store_data.Promo2SinceWeek.fillna(0, inplace=True)
store_data.Promo2SinceYear.fillna(0, inplace=True)
store_data.PromoInterval.fillna(0, inplace=True)
# store_data.query('PromoInterval ==0 & Promo2 == 0', engine='python')

In [17]:
store_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1115 non-null   float64
 4   CompetitionOpenSinceMonth  1115 non-null   float64
 5   CompetitionOpenSinceYear   1115 non-null   float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            1115 non-null   float64
 8   Promo2SinceYear            1115 non-null   float64
 9   PromoInterval              1115 non-null   object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


### - Handling Missing data (NaN) Train Dataset

In [18]:
test_data.isna().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

### The Open column is not continuous, then filling the missing values with MOde method

In [19]:
test_data.Open.fillna(test_data.Open.mode()[0],inplace=True)
# test_data.sort_values(by='Open')

## Feature extraction and Generation in Train Dataset
### Adding the following features,
 - weekdays
 - weekends 
 - number of days to holidays
 - Number of days after holiday
 - Beginning of month, mid month and ending of month


In [20]:
train_data.Date = pd.to_datetime(train_data.Date)
try:
    train_data['Year'] = train_data.Date.dt.year
    train_data['Month'] = train_data.Date.dt.month
    train_data['Day'] = train_data.Date.dt.day
    train_data['WeekOfYear'] = train_data.Date.dt.weekofyear
    train_data['Weekofmonth'] = train_data.Date.dt.day//7 + 1

except Exception as e:
    print(e)
    logging.exception(f"Exception occured in extracting Features and adding to dataset, Exception:{e}")


In [35]:
train_data.sample(5)
# test_data.sort_values(by='Date')

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,Weekofmonth
655154,320,4,2013-11-21,14897,2131,1,1,0,0,2013,11,21,47,4
123239,590,7,2015-04-12,0,0,0,0,0,0,2015,4,12,15,2
178380,1096,7,2015-02-22,0,0,0,0,0,0,2015,2,22,8,4
427194,935,6,2014-06-14,3733,403,1,0,0,0,2014,6,14,24,3
560150,91,5,2014-02-14,6347,744,1,0,0,0,2014,2,14,7,3


### Finding the holiday dates

In [22]:
try:
    holiday_dates = train_data[(train_data['StateHoliday'] != '0') & (train_data['SchoolHoliday'] != '0')]
    date_list = holiday_dates.Date.dt.strftime("%y,%m,%d").unique()
    list_date = []
    for date_item in date_list:
        list_date.append(datetime.strptime(date_item, '%y,%m,%d'))

except Exception as e:
    print(e)
    logging.exception(f"Exception occured in extracting days from/to holiday Features and adding to dataset, Exception:{e}")

# list_date

In [23]:
from datetime import datetime
def nearest_date(items,pivot):
    res = min(items, key=lambda sub: abs(sub - pivot))
   
    timedelta = abs(res - pivot)
   
    return timedelta.days

# items = list_date
# pivot = datetime(2014, 3, 21, 0, 0)
# print(nearest_date(items, pivot))


2


In [51]:
df = pd.DataFrame()
try:

    date_list1 = train_data.Date.dt.strftime("%y,%m,%d")
    list2 = []
    for list1 in date_list1:
        list2.append(nearest_date(list_date, datetime.strptime(list1, '%y,%m,%d')))
except Exception as e:
    print(e)
    logging.exception(f"Exception occured in extracting days from/to holiday Features, Exception:{e}")

In [53]:
df = pd.DataFrame(list(zip(list2)), columns =['days'])
df.head()

,days
0,57
1,57
2,57
3,57
4,57


In [54]:
# Adding days to/from holiday
try:

    train_data['days_to_from_hol'] = df['days']
    train_data.sample(4)
    logging.info(f"Adding days_to/from holiday to train dataset, successfully")
except Exception as e:
    print(e)
    logging.exception(f"Exception occured while adding days_to/from holiday to train dataset, Exception:{e}")

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,Weekofmonth,days_to_from_hol
146427,363,7,2015-03-22,0,0,0,0,0,0,2015,3,22,12,4,12
281183,1025,5,2014-11-14,7506,836,1,1,0,0,2014,11,14,46,3,5
785074,900,7,2013-07-28,0,0,0,0,0,0,2013,7,28,30,5,18
514789,445,4,2014-03-27,4417,757,1,0,0,0,2014,3,27,13,4,8
